In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from pytorch_lightning import LightningModule


class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)


class SimpsonsNet(LightningModule):
    def __init__(self):
        super(SimpsonsNet, self).__init__()
        
        self.sequential = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),
            Flatten(),
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=False),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=False),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        out = self.sequential(x)
        return F.log_softmax(out, dim=-1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        self.log('Training Loss', loss)
        return loss

    def _evaluate(self, batch, batch_idx, stage=None):
        x, y = batch
        out = self.forward(x)
        logits = F.log_softmax(out, dim=-1)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=-1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f'{stage}_loss', loss, prog_bar=True)
            self.log(f'{stage}_acc', acc, prog_bar=True)

        return loss, acc

    def validation_step(self, batch, batch_idx):
        return self._evaluate(batch, batch_idx, 'val')[0]

    def test_step(self, batch, batch_idx):
        loss, acc = self._evaluate(batch, batch_idx, 'test')
        self.log_dict({'test_loss': loss, 'test_acc': acc})

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=2e-3)

In [7]:
model = SimpsonsNet()

In [8]:
model

SimpsonsNet(
  (sequential): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten()
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=4096, out_features=1024, bias=True)
    (9): ReLU()
    (10): Linear(in_features=1024, out_features=512, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.1, inplace=False)
    (13): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [ ]:
# import torch.nn as nn

# from pytorch_lightning import LightningModule


# class MiniVGG(LightningModule):
#     def __init__(self, num_classes=10):
#         super(MiniVGG, self).__init__()
        
#         self.features = [
#             nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
#             nn.BatchNorm2d(num_features=32),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(num_features=64),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(num_features=64),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(num_features=64),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         ]
        
#         self.avgpool = nn.AdaptiveAvgPool2d(output_size=(7, 7))
        
#         self.classifier = nn.Sequential([
#             nn.Linear(64 * 7 * 7, 1024),
#             nn.ReLU(inplace=True)
#             nn.Dropout(),
#             nn.Linear(1024, 1024),
#             nn.ReLU(inplace=True)
#             nn.Dropout(),
#             nn.Linear(1024, num_classes)
#         ])
        
#     def forward(self, x):
#         pass
    
#     def training_step():
#         pass
    
#     def validation_step():
#         pass
    
#     def validation_epoch_end(self, outputs):
#         # OPTIONAL
#         avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#         tensorboard_logs = {'val_loss': avg_loss}
#         return {'val_loss': avg_loss, 'log': tensorboard_logs}

#     def test_step(self, batch, batch_nb):
#         # OPTIONAL
#         x, y = batch
#         y_hat = self(x)
#         return {'test_loss': F.cross_entropy(y_hat, y)}

#     def test_epoch_end(self, outputs):
#         # OPTIONAL
#         avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
#         logs = {'test_loss': avg_loss}
#         return {'test_loss': avg_loss, 'log': logs, 'progress_bar': logs}

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=2e-3)